In [1]:
import cv2, numpy, os
haar_file = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)
dataset = 'dataset'
print('Training....')

(images,labels,names,id) = ([],[],{},0)

for (subdirs, dirs, files) in os.walk(dataset):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(dataset, subdir)
        for filename in os.listdir(subjectpath):
            path = subjectpath + '/' + filename
            label = id
            images.append(cv2.imread(path,0))
            labels.append(int(label))
        id = id + 1

(images,labels) = [numpy.array(lis) for lis in [images,labels]]
print(images, labels)
(width,height) = (130,100)

model = cv2.face.LBPHFaceRecognizer_create()
#model = cv2.face.FisherFaceRecognizer_create()

model.train(images, labels)

webcam = cv2.VideoCapture(0)
cnt = 0

while True:
    (_,im) = webcam.read()
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(im,(x,y),(x+w,y+h),(255,255,0),2)
        face = gray[y:y+h,x:x+w]
        face_resize = cv2.resize(face,(width,height))

        prediction = model.predict(face_resize)
        cv2.rectangle(im,(x,y),(x+w,y+h),(255,255,0),2)
        if prediction[1] < 800:
            cv2.putText(im,'%s - %.0f' % (names[prediction[0]],prediction[1]),(x-10,y-10),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255))
            print(names[prediction[0]])
            cnt = 0
        else:
            cnt += 1
            cv2.putText(im,'Unknown',(x-10,y-10),cv2.FONT_HERSHEY_PLAIN,1,(0,255,0))
            if cnt > 100:
                print("Unknown Person")
                cv2.imwrite("Unknown.jpg",im)
                cnt = 0
    cv2.imshow('FaceRecognition',im)
    key = cv2.waitKey(10)
    if key == 27:
        break
webcam.release()


Training....
[[[145 144 141 ... 185 183 183]
  [148 147 145 ... 173 174 174]
  [150 150 149 ... 162 164 164]
  ...
  [184 186 188 ... 182 184 184]
  [184 186 188 ... 183 184 184]
  [184 185 188 ... 183 183 183]]

 [[146 149 149 ...  96 111  90]
  [149 154 144 ... 108 116  86]
  [146 145 149 ... 100 104  89]
  ...
  [202 205 207 ... 238 238 238]
  [206 199 193 ... 233 238 238]
  [200 202 180 ... 237 236 236]]

 [[158 159 154 ... 158 149 145]
  [161 157 150 ... 154 154 156]
  [152 145 154 ... 152 148 146]
  ...
  [212 207 210 ... 160 160 166]
  [210 210 210 ... 171 170 175]
  [209 213 210 ... 184 178 182]]

 ...

 [[136 135 133 ... 153 148 150]
  [155 145 137 ... 149 152 152]
  [150 137 136 ... 159 146 146]
  ...
  [179 106 114 ... 202 195 188]
  [155  86 190 ... 205 187 181]
  [101 135 187 ... 207 191 185]]

 [[149 150 148 ... 153 144 136]
  [148 147 145 ... 148 161 146]
  [149 149 150 ... 150 151 147]
  ...
  [184 187 183 ... 232 230 230]
  [180 188 147 ... 233 232 232]
  [191 166 113 